In [1]:
import utils
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from scipy import linalg
import numpy as np

In [ ]:
torch.manual_seed(1992)
np.random.seed(1992)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

##### PyTorch uses tensors in format [samples,  channels, size]

In [2]:
trainX, trainy, _, _ = utils.load_dataset()
trainX = torch.tensor(trainX, dtype=torch.float32)[0, :, 0].view(1, 1, 128)
trainy = torch.tensor(trainy)

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 1) (2947, 128, 9) (2947, 1)


#### Utils

In [15]:
trainX.shape

torch.Size([1, 1, 128])

In [3]:
def build_tensor(size, kernel_size, identity=False):
    coords = []
    vals = []
    s = size
    if identity:
        s = size - 1
    for k in range(s):
        for j in range(kernel_size):
            i = k - j + 1
            if k in range(s) and i in range(s) and j in range(kernel_size):
                coords.append([k, i, j])
                vals.append(1)

    if identity:
        coords.append([s, s, kernel_size])
        vals.append(1)
        i = torch.LongTensor(coords)
        v = torch.FloatTensor(vals)
        return torch.sparse.FloatTensor(i.t(), v, torch.Size([size, size, kernel_size+1]))        
                
    i = torch.LongTensor(coords)
    v = torch.FloatTensor(vals)
    return torch.sparse.FloatTensor(i.t(), v, torch.Size([size, size, kernel_size]))

In [22]:
build_tensor(5, 3, True).to_dense()

tensor([[[0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 1., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 1., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 1., 0.],
         [0., 1., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 1.]]])

## Basic conv1d module

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        ### PRE-DENSE LAYERS ###
        self.conv = nn.Conv1d(1, 1, 3, padding=1, bias=False)

    def forward(self, x):
        x = self.conv(x)
        return x
    
    def get_weights(self):
        return self.conv.weight[0]

In [5]:
model = Net()
model.forward(trainX)

tensor([[[0.8485, 0.5086, 0.5030, 0.5044, 0.5049, 0.5008, 0.5041, 0.5031,
          0.5035, 0.5051, 0.5032, 0.5046, 0.5036, 0.5038, 0.5046, 0.5032,
          0.5046, 0.5042, 0.5048, 0.5055, 0.5039, 0.5050, 0.5046, 0.5039,
          0.5039, 0.5033, 0.5043, 0.5040, 0.5037, 0.5040, 0.5030, 0.5029,
          0.5040, 0.5048, 0.5039, 0.5035, 0.5044, 0.5039, 0.5042, 0.5053,
          0.5044, 0.5031, 0.5010, 0.5008, 0.5035, 0.5052, 0.5041, 0.5021,
          0.5020, 0.5042, 0.5056, 0.5045, 0.5024, 0.5033, 0.5061, 0.5053,
          0.5045, 0.5057, 0.5042, 0.5018, 0.5018, 0.5022, 0.5017, 0.5033,
          0.5065, 0.5059, 0.5038, 0.5047, 0.5049, 0.5039, 0.5036, 0.5032,
          0.5029, 0.5031, 0.5033, 0.5034, 0.5035, 0.5034, 0.5027, 0.5028,
          0.5038, 0.5034, 0.5028, 0.5036, 0.5033, 0.5035, 0.5046, 0.5042,
          0.5043, 0.5051, 0.5054, 0.5054, 0.5045, 0.5032, 0.5029, 0.5029,
          0.5014, 0.5029, 0.5078, 0.5070, 0.5034, 0.5039, 0.5038, 0.5033,
          0.5044, 0.5039, 0.5037, 0.50

## Basic custom convolution module

In [23]:
class CustomNet(nn.Module):
    def __init__(self, w):
        super(CustomNet, self).__init__()
        self.w = w.T
        self.T = build_tensor(128, 3).to_dense()
        
    def forward(self, x):
        self.weights = torch.matmul(self.T, self.w)[:, :, 0]
        x = torch.matmul(x, self.weights)
        return x

In [24]:
customModel = CustomNet(model.get_weights())
customModel.forward(trainX)

tensor([[[0.8485, 0.5086, 0.5030, 0.5044, 0.5049, 0.5008, 0.5041, 0.5031,
          0.5035, 0.5051, 0.5032, 0.5046, 0.5036, 0.5038, 0.5046, 0.5032,
          0.5046, 0.5042, 0.5048, 0.5055, 0.5039, 0.5050, 0.5046, 0.5039,
          0.5039, 0.5033, 0.5043, 0.5040, 0.5037, 0.5040, 0.5030, 0.5029,
          0.5040, 0.5048, 0.5039, 0.5035, 0.5044, 0.5039, 0.5042, 0.5053,
          0.5044, 0.5031, 0.5010, 0.5008, 0.5035, 0.5052, 0.5041, 0.5021,
          0.5020, 0.5042, 0.5056, 0.5045, 0.5024, 0.5033, 0.5061, 0.5053,
          0.5045, 0.5057, 0.5042, 0.5018, 0.5018, 0.5022, 0.5017, 0.5033,
          0.5065, 0.5059, 0.5038, 0.5047, 0.5049, 0.5039, 0.5036, 0.5032,
          0.5029, 0.5031, 0.5033, 0.5034, 0.5035, 0.5034, 0.5027, 0.5028,
          0.5038, 0.5034, 0.5028, 0.5036, 0.5033, 0.5035, 0.5046, 0.5042,
          0.5043, 0.5051, 0.5054, 0.5054, 0.5045, 0.5032, 0.5029, 0.5029,
          0.5014, 0.5029, 0.5078, 0.5070, 0.5034, 0.5039, 0.5038, 0.5033,
          0.5044, 0.5039, 0.5037, 0.50

In [25]:
torch.all(model.forward(trainX) == customModel.forward(trainX))

tensor(True)

## CustomConvolution adding "identity element"

In [9]:
class CustomNetConv(nn.Module):
    def __init__(self, w):
        super(CustomNetConv, self).__init__()
        self.weight = nn.Parameter(w.T)
        self.weight.
        self.T = build_tensor(129, 3, True).to_dense()
    
    def forward(self, x):
        self.W = torch.matmul(self.T, torch.cat([self.weight, torch.tensor([[1]])]))[:, :, 0]
        #### TODO ajustar ordem: torch.matmul(torch.matmul(self.B, self.W), x)
        x = torch.matmul(x, self.W)
        return x

In [29]:
X = torch.cat([trainX[0, 0, :], torch.tensor([1])]).view(1, 1, 129)
customModelConv = CustomNetConv(model.get_weights())
customModelConv.forward(X)

tensor([[[0.8485, 0.5086, 0.5030, 0.5044, 0.5049, 0.5008, 0.5041, 0.5031,
          0.5035, 0.5051, 0.5032, 0.5046, 0.5036, 0.5038, 0.5046, 0.5032,
          0.5046, 0.5042, 0.5048, 0.5055, 0.5039, 0.5050, 0.5046, 0.5039,
          0.5039, 0.5033, 0.5043, 0.5040, 0.5037, 0.5040, 0.5030, 0.5029,
          0.5040, 0.5048, 0.5039, 0.5035, 0.5044, 0.5039, 0.5042, 0.5053,
          0.5044, 0.5031, 0.5010, 0.5008, 0.5035, 0.5052, 0.5041, 0.5021,
          0.5020, 0.5042, 0.5056, 0.5045, 0.5024, 0.5033, 0.5061, 0.5053,
          0.5045, 0.5057, 0.5042, 0.5018, 0.5018, 0.5022, 0.5017, 0.5033,
          0.5065, 0.5059, 0.5038, 0.5047, 0.5049, 0.5039, 0.5036, 0.5032,
          0.5029, 0.5031, 0.5033, 0.5034, 0.5035, 0.5034, 0.5027, 0.5028,
          0.5038, 0.5034, 0.5028, 0.5036, 0.5033, 0.5035, 0.5046, 0.5042,
          0.5043, 0.5051, 0.5054, 0.5054, 0.5045, 0.5032, 0.5029, 0.5029,
          0.5014, 0.5029, 0.5078, 0.5070, 0.5034, 0.5039, 0.5038, 0.5033,
          0.5044, 0.5039, 0.5037, 0.50

In [27]:
torch.all(customModelConv.forward(X)[:, :, :-1] == model.forward(trainX))

tensor(True)

## CustomConvolution adding "identity element" and bias

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        ### PRE-DENSE LAYERS ###
        self.conv1 = nn.Conv1d(1, 1, 3, padding=1, bias=False)
        self.activ1 = nn.Tanh()
        self.conv2 = nn.Conv1d(1, 1, 3, padding=1, bias=False)
        self.activ2 = nn.Tanh()
        ### DENSE LAYERS ###
        self.flat = nn.Flatten(1,-1)
        self.dense1 = nn.Linear(128, 100)
        self.activ3 = nn.Tanh()
        self.dense2 = nn.Linear(100, 6)
        self.activ4 = nn.Softmax()

    def forward(self, x):
        x = self.conv1(x)
        x = self.activ1(x)
        x = self.conv2(x)
        x = self.activ2(x)
        x = self.flat(x)
        x = self.dense1(x)
        x = self.activ3(x)
        x = self.dense2(x)
        x = self.activ4(x)
        return x
    
    def get_weights(self):
        return self.conv.weight[0]